In [5]:
import hashlib
from random import randint , randrange
from math import gcd
import sympy as sn

# Exercice 1

# RSA

In [2]:
def generer(nb) :
    #generer deux nombre premier distinctes de nb bits
    p,q = 1,1
    while p==q:  
        p=sn.randprime( 2**nb , 2**(nb+1) -1 )
        q=sn.randprime( 2**nb , 2**(nb+1) -1 )
        
        
    #caluler n
    n=q*p
    
    #calculer fi(n)
    fi = (q-1)*(p-1)
    
    #trouver un entier e telque pgcd( e , fi(n) ) = 1
    e=pow(2,16)+1
    
    #calculer d
    d= pow(e,-1 , fi )
    
    return (n,e), (n,d)

In [3]:
def chiffrer( text , cle_pr ) :
    #calculer hach
    hashed_string = hashlib.sha256(text.encode('utf-8')).hexdigest()
    
    #calculer signature
    s=  pow( int(hashed_string , 16 ) , cle_pr[1] , cle_pr[0] )
    
    return s

In [4]:
 def dechiffrer( text , signature , cle_pub ) :
    #calculer hach de text
    hashed_string = hashlib.sha256(text.encode('utf-8')).hexdigest()
    
    #dechiffrer la sigature
    h = pow( signature , cle_pub[1] , cle_pub[0])
    
    return h==int(hashed_string , 16 )

In [5]:
text = 'Hello word'

cle_pub , cle_pr = generer(1024)

In [6]:
s = chiffrer(text , cle_pr)

In [7]:
print('La signature numériue est : ' , s )

La signature numériue est :  87088358710130393013729435620986598972097664808153868271470704976602603149470067957360046675613413507597646232990293189848967229793617829872130967599693485883855486697714008271795451749875792114805455668370405667246938874363950980530078124583342582651814323121642406706136567317436803348461444740972604835133952339378614923577857266874744195229489547513132233959345136321198780215229076914262962934991666194506931735720442320861411158711088794597815287799175509682575278174750073099856985631269623023244714479888395377831951575516555329368405676163840977914124433246959228279305422824996479148737350032876871576701823


In [8]:
dechiffrer( text , s , cle_pub)

True

# CRT-RSA

In [9]:
def generer_crt(nb) :
    
    #generer 2 nombres premiers
    p=sn.randprime( 2**nb , 2**(nb+1) -1 )
    q=sn.randprime( 2**nb , 2**(nb+1) -1 )
    
    #calculer n = p*q
    n=p*q
    
    
    e=pow(2,16)+1
    
    dp = pow ( e , -1 , p-1 )
    dq = pow ( e , -1 , q-1 )
    
    
    return (n,e)  , (p, q, dp, dq) 

In [10]:
def chiffrer( text , cle_pr ) :
    #calculer hach
    hashed_string = hashlib.sha256(text.encode('utf-8')).hexdigest()
    #hash en decimal
    i=int(hashed_string , 16 )
    
    #calculer signature
    mp = pow( i, cle_pr[2] , cle_pr[0] )
    mq = pow( i, cle_pr[3] , cle_pr[1] )
        
    q_inv = pow(cle_pr[1] , -1 , cle_pr[0])
        
    h = q_inv * (mp - mq)
        
    m = mq + cle_pr[1]*h
    
    return m

In [11]:
cle_pub , cle_pr = generer_crt(1024)

In [12]:
text = 'hello'

In [13]:
s = chiffrer(text ,cle_pr)
s

1184460188628075804495210779691015907668101144447556311978987779820242197877717228621236623738380294496687020885496308976601205845777321599471155785642732351371736848967137742338215928586760803904654319669345556269368636772940491731015612929667085389508104099851703962100684611194376851027174244649131022159589741712311927659312020928809933184616739720784215758241949716300496897818480205278647763203613300106752129215712627127450865488671189838082173732638257245305761129922395727188258779002167773697522001618656524278528773908997253966076542964784580125942817303578551515234367659048914217756446296332526954260307577748642347789010187019637785018892523351643188410288564134791583239375964106532761902982655844340330722975703197742236294473570302422062624037504848116193368311225082946138409566141173098556123515268016023309001669829027183420994824766245274529770063701769155320274188852086546074979428712040111857059902711

In [14]:
dechiffrer( text , s , cle_pub)

True

# DSA

In [10]:
def parameter_generation():

    # primes of 8 bits in length in binary
    p=0
    while ( sn.isprime(p) == False ) :
        k = randint(2**10 , 2**11 - 1 )
        q= sn.randprime( 2**10, 2**11 - 1)
        p=k*q+1
         
    a=1
    while(a==1):
        h = randint(1 , p-1 )
        a=pow(h,int((p-1)/q) ,p)
       
    d=randint(1,q-1)

    b=pow(a,d,p)
       
    return(p,q,a,b) , (d)

In [11]:
def signature(text,p,q,a,d):
    hashed = hashlib.sha256(text.encode('utf-8')).hexdigest()
    i =int(hashed , 16 ) 
     
    k=randint(1,q-1)
    r=pow(a,k,p)%q
    print('R --> ' , r)
    
    s=(pow(k,-1 , q)*(i+(d*r)))%q
    print('S --> ' , s)
    
    return(r,s)

In [12]:
def verification(text,p,q,a,r,s,y):
    
    hashed = hashlib.sha256(text.encode('utf-8')).hexdigest()
    i =int(hashed , 16 ) 

    # computing w
    w=pow(s,-1 , q)
    

    # computing u1, u2 and v
    u1=(i*w)%q 
    u2=(r*w)%q
    v=(pow(a,u1,p)*pow(y,u2 ,p)) % p %q
    

    print('V --> ' , v)
    return v == r 


In [13]:
(p,q,a,b) , (d) =parameter_generation()

text = 'hello'
components=signature(text,p,q,a,d)

verification(text,p,q,a,components[0],components[1],b)

R -->  1449
S -->  724
V -->  1449


True